In [1]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import albumentations as A
from efficientnet_pytorch import EfficientNet
from pprint import pprint
import timm
import random
from torch.autograd import Variable

import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

/opt/conda/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels = pd.read_csv("/data/IEEE_BigData/cls_crop/total_labels_0915_01.csv")

#labels_semi_1 = pd.read_csv("/data/IEEE_BigData/cls_crop/VOC_cycle_semi_v1.csv")

In [3]:
labels

,File,Class,min_x,min_y,max_x,max_y,v_cls,semi_v1,semi_v1_conf
0,/data/IEEE_BigData/cls_crop/VOC_cycle/0000001.jpg,-1,162,471,241,513,0,7,0.999489
1,/data/IEEE_BigData/cls_crop/VOC_cycle/0000012.jpg,-1,451,459,513,516,0,7,0.998670
2,/data/IEEE_BigData/cls_crop/VOC_cycle/0000014.jpg,-1,1142,428,1166,453,0,6,0.973409
3,/data/IEEE_BigData/cls_crop/VOC_cycle/0000015.jpg,-1,1152,441,1186,473,0,6,0.988077
4,/data/IEEE_BigData/cls_crop/VOC_cycle/0000016.jpg,-1,1270,449,1328,499,0,6,0.958759
...,...,...,...,...,...,...,...,...,...
3313,/data/IEEE_BigData/cls_crop/VOC_cycle/0015873.jpg,-1,179,443,321,551,0,7,0.997142
3314,/data/IEEE_BigData/cls_crop/VOC_cycle/0015880.jpg,-1,1560,434,1617,464,0,7,0.996551
3315,/data/IEEE_BigData/cls_crop/VOC_cycle/0015881.jpg,-1,1561,433,1618,464,0,7,0.995811
3316,/data/IEEE_BigData/cls_crop/VOC_cycle/0015882.jpg,-1,1560,434,1617,464,0,7,0.994871


In [3]:
#labels_semi_1["Class"] = labels_semi_1["semi_v1"]

In [4]:
labels

,File,Class,min_x,min_y,max_x,max_y,v_cls
0,/data/IEEE_BigData/crop_train/0000001.jpg,11,0.553646,0.532407,0.592187,0.611111,3
1,/data/IEEE_BigData/crop_train/0000002.jpg,0,0.608333,0.405556,0.769792,0.606481,0
2,/data/IEEE_BigData/crop_train/0000003.jpg,0,0.600521,0.470370,0.697396,0.602778,0
3,/data/IEEE_BigData/crop_train/0000004.jpg,0,0.327604,0.454630,0.411979,0.586111,0
4,/data/IEEE_BigData/crop_train/0000005.jpg,0,0.678646,0.347222,0.999479,1.000000,0
...,...,...,...,...,...,...,...
209623,/data/IEEE_BigData/cls_crop/gan_result/0104810...,2,0.512500,0.508333,0.538542,0.538889,0
209624,/data/IEEE_BigData/cls_crop/gan_result/0104811...,2,0.512500,0.508333,0.538542,0.538889,0
209625,/data/IEEE_BigData/cls_crop/gan_result/0104812...,2,0.512500,0.508333,0.538542,0.538889,0
209626,/data/IEEE_BigData/cls_crop/gan_result/0104813...,2,0.512500,0.508333,0.538542,0.538889,0


In [5]:
decoder = {"0" : "car_back",
           "1" : "car_side",
           "2" : "car_front",
           "3" : "truck_back",
           "4" : "truck_side",
           "5" : "truck_front",
           "6" : "motorcycle_back",
           "7" : "motorcycle_side",
           "8" : "motorcycle_front",
           "9" : "bicycle_back",
           "10" : "bicycle_side",
           "11" : "bicycle_front"}

# Total

In [6]:
class TotalDataset(Dataset):
    def __init__(self, labels=None, transformer=None, mode="train"):
        self.transformer = transformer
        self.mode = mode
        if self.mode == "train" :
            self.labels = labels
        elif self.mode == "val" :
            self.labels = labels
            
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self, i):
        img = cv2.imread(self.labels["File"].iloc[i]).astype(np.float32)/255
        img = cv2.resize(img, dsize=(384,384))
        if self.mode == "train":
            if self.transformer != None:
                img = self.transformer(image=img)
                img = np.transpose(img["image"], (2,0,1))
            else:
                img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32),
                "label" : torch.tensor(self.labels["Class"].iloc[i], dtype=torch.long)
            }
        elif self.mode == "val":
            img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32),
                "label" : torch.tensor(self.labels["Class"].iloc[i], dtype=torch.long)
            }
        
        else:
            img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32)
            }

In [7]:
def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        pred = torch.argmax(output, axis=1)
        acc = torch.sum(pred == label)
        return loss, acc
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
            pred = torch.argmax(output, axis=1)
            acc = torch.sum(pred == label)
            
        return loss, acc

In [8]:
class LabelSmoothingLoss(nn.Module):
    """
    With label smoothing,
    KL-divergence between q_{smoothed ground truth prob.}(w)
    and p_{prob. computed by model}(w) is minimized.
    """
    def __init__(self, label_smoothing, tgt_vocab_size, ignore_index=-100):
        assert 0.0 < label_smoothing <= 1.0
        self.ignore_index = ignore_index
        super(LabelSmoothingLoss, self).__init__()

        smoothing_value = label_smoothing / (tgt_vocab_size - 2)
        one_hot = torch.full((tgt_vocab_size,), smoothing_value)
        one_hot[self.ignore_index] = 0
        self.register_buffer('one_hot', one_hot.unsqueeze(0))

        self.confidence = 1.0 - label_smoothing

    def forward(self, output, target):
        """
        output (FloatTensor): batch_size x n_classes
        target (LongTensor): batch_size
        """
        model_prob = self.one_hot.repeat(target.size(0), 1)
        model_prob.scatter_(1, target.unsqueeze(1), self.confidence)
        model_prob.masked_fill_((target == self.ignore_index).unsqueeze(1), 0)

        return F.kl_div(output, model_prob, reduction='sum')

In [9]:
FOLDS = 5
skf = StratifiedKFold(n_splits=FOLDS, random_state=42, shuffle=True)

device = torch.device("cuda:0")
dropout_rate = 0.1
class_num = 12
learning_rate = 1e-4
BATCH_SIZE = 48
EPOCHS = 25
MODELS = 'efficientnet-b7'
#MODELS = 'regnety_040'
save_path = f"./models/Py_{MODELS}_GAN_0922_{EPOCHS}"

folder_train_idx = 0
folder_val_idx = 0

n=0

for train_idx, val_idx in skf.split(range(labels.shape[0]), labels["Class"]):
    
#    train_idx = np.array(list(train_idx) + list(range(labels.shape[0], labels.shape[0] + labels_semi_1.shape[0])))
    
#    labels = pd.concat((labels, labels_semi_1))
#    labels = labels.reset_index().drop(columns=["index"])
    
    n = n + 1
    
    if (n == 1) or (n == 2) or (n == 3) or (n == 4):
        continue
    
    albumentations_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.GaussianBlur(),
        A.ShiftScaleRotate(),
        A.GaussNoise(),
        A.RandomGamma(),
    ])
    
    model = EfficientNet.from_pretrained(MODELS, advprop=True, num_classes=class_num)
#    model = torch.load(f"./models/Py_efficientnet-b7_GAN_30_1.pt")
#    model = timm.create_model(MODELS, pretrained=True, num_classes=class_num)
#    model._dropout.p = dropout_rate
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, patience=5)
    criterion = nn.CrossEntropyLoss()

    train_dataset = TotalDataset(labels.iloc[train_idx], transformer=albumentations_transform)
    val_dataset = TotalDataset(labels.iloc[val_idx], transformer=albumentations_transform, mode="val")

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=False)

    sample_batch = next(iter(train_dataloader))

    loss_plot, val_loss_plot = [], []

    for epoch in range(EPOCHS):
        total_loss, total_val_loss = 0, 0
        total_acc, total_val_acc = 0, 0

        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_acc = train_step(batch_item, epoch, batch, training)
            total_loss += batch_loss
            total_acc += batch_acc

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Total Loss' : '{:06f}'.format(total_loss/(batch+1)),
                'Total Acc' : '{:06f}'.format((total_acc/((batch+1) * 48)) * 100)
            })
        loss_plot.append(total_loss/(batch+1))

        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_acc = train_step(batch_item, epoch, batch, training)
            total_val_loss += batch_loss
            total_val_acc += batch_acc

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
                'Total Val Acc' : '{:06f}'.format((total_val_acc/((batch+1) * 48)) * 100)
            })
        val_loss_plot.append(total_val_loss/(batch+1))
        scheduler.step(total_val_loss/(batch+1))

        if np.min(val_loss_plot) == val_loss_plot[-1]:
            torch.save(model, save_path + f"_{n}.pt")
            print("## Model Save")

Loaded pretrained weights for efficientnet-b7


3494it [1:03:55,  1.10s/it, Epoch=1, Loss=0.303576, Total Loss=0.622655, Total Acc=78.567429]
874it [07:25,  1.96it/s, Epoch=1, Val Loss=0.001469, Total Val Loss=0.024365, Total Val Acc=99.191933]


## Model Save


3494it [1:03:51,  1.10s/it, Epoch=2, Loss=0.491104, Total Loss=0.382309, Total Acc=86.520943]
874it [07:25,  1.96it/s, Epoch=2, Val Loss=0.005843, Total Val Loss=0.018708, Total Val Acc=99.432686]


## Model Save


...

## Model Save


3494it [1:04:06,  1.10s/it, Epoch=23, Loss=0.166500, Total Loss=0.178787, Total Acc=93.921730]
874it [07:31,  1.94it/s, Epoch=23, Val Loss=0.000082, Total Val Loss=0.003346, Total Val Acc=99.830765]


## Model Save


3494it [1:04:02,  1.10s/it, Epoch=24, Loss=0.063551, Total Loss=0.178029, Total Acc=93.916359]


OSError: [Errno 12] Cannot allocate memory

# Test

# Car

# Moto

# Truck

# Cycle